In [2]:
%pip install aiogram


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'a:\Users\Документы\Source\Repos\Python_Projects\OBS_TBot\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.types import Message
from aiogram.types import ParseMode
from aiogram.utils import executor
import asyncio
from datetime import datetime
from zoneinfo import ZoneInfo  # работает только с Python 3.9+
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()
moscow_time = datetime.now(ZoneInfo("Europe/Moscow"))
print(f"Московское время: {moscow_time}")

TOKEN = BOT_TOKEN = os.getenv("BOT_TOKEN")

bot = Bot(token=TOKEN, parse_mode=ParseMode.HTML)
dp = Dispatcher(bot)

# bot = Bot(
#     token=TOKEN,
#     default=DefaultBotProperties(parse_mode=ParseMode.HTML)
# )

# dp = Dispatcher()

Московское время: 2025-07-31 17:55:58.326177+03:00


In [3]:
@dp.message_handler()
async def reminder_handler(message: types.Message):
    text = message.text.strip().lower()
    print(f"[{datetime.now()}] Получено сообщение: {text}")

    if text.startswith("через"):
        try:
            _, minutes_str, reminder_text = text.split(" ", 2)
            delay = int(minutes_str)

        except Exception as e:
            print(f"[{datetime.now()}] Ошибка при обработке 'через': {e}")
            await message.answer("⚠️ Формат: через <минуты> напомни <текст>")
            return
    else:
        try:
            parts = text.split(" ", 3)
            if len(parts) < 4:
                raise ValueError("Недостаточно аргументов")
            date_part, time_part, _, reminder_text = parts
            reminder_dt = datetime.strptime(f"{date_part} {time_part}", "%d.%m.%Y %H:%M")
            delay = (reminder_dt - datetime.now()).total_seconds()
            if delay <= 0:
                print(f"[{datetime.now()}] Введено прошедшее время: {reminder_dt}")
                await message.answer("❗ Время уже прошло!")
                return
        except Exception as e:
            print(f"[{datetime.now()}] Ошибка при обработке даты: {e}")
            await message.answer("⚠️ Формат: <дд.мм.гггг> <чч:мм> напомни <текст>")
            return

    print(f"[{datetime.now()}] Установлено напоминание через {int(delay)} секунд: {reminder_text}")
    await message.answer(f"⏰ Напоминание установлено!")
    asyncio.create_task(set_reminder(message.chat.id, delay, reminder_text))

async def set_reminder(chat_id, delay, text):
    print(f"[{datetime.now()}] Ждём {int(delay)} секунд для чата {chat_id} — {text}")
    await asyncio.sleep(delay)
    print(f"[{datetime.now()}] Напоминание сработало для чата {chat_id}: {text}")
    await bot.send_message(chat_id, f"🔔 Напоминаю: <b>{text}</b>")


In [7]:
# Запускаем polling (один раз)
polling_task = asyncio.create_task(dp.start_polling(bot))


In [8]:
polling_task.cancel()

try:
    await polling_task
except asyncio.CancelledError:
    print("Polling успешно остановлен")


RuntimeError: Polling already started